In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (15,5)
plt.rcParams['axes.grid'] = False
import seaborn as sns
sns.set_style("whitegrid", {'axes.grid' : False})

In [4]:
from Feature_Selection import *

In [5]:
data = get_data('NIFc1', 'D')

#Add Forward Returns
freturn_types = ["FReturn", "MaxFReturn", "MinFReturn"]
lookforwards = [1,2,3,5,7,10,15,20,30,45,60]
for i in range(1,max(lookforwards)):
    data[f"FReturn{i}"] = data["Close"].shift(-i)/data["Close"]-1
data.dropna(inplace=True)
for lookforward in lookforwards:
    data[f"MaxFReturn{lookforward}"] = data[[f"FReturn{i}" for i in range(1,lookforward)]].max( axis=1)
    data[f"MinFReturn{lookforward}"] = data[[f"FReturn{i}" for i in range(1,lookforward)]].min( axis=1)

data = data.drop(columns=[f"FReturn{i}" for i in range(1,max(lookforwards)) if i not in lookforwards])


In [6]:
#Feature space
lookbacks = [2,3,5,7,10,15,21,30,45,50,63,75,90,100,126,150,175,200,225,252]
f_functions = [add_fisher, add_inverse_fisher, add_constance_brown, add_BBANDS, add_DEMA, add_EMA, add_HT_TRENDLINE, add_KAMA, add_MA, add_MAMA, add_MIDPOINT, add_MIDPRICE, add_SAR, ]
g_functions = [x, max_over_lookback, min_over_lookback, sma, shift]

#Creating combinations for NMI calculation
params = [f_functions, lookbacks, g_functions, lookbacks, freturn_types, lookforwards]
inputs = list(itertools.product(*params))

In [9]:
for input in inputs:
    [f, flb, g, glb, frtype, lf] = input
    X = f([data, flb]).iloc[flb-1:].reset_index(drop=True)
    col_name = X.columns[-1]
    X = g(f([data, flb]).iloc[flb-1:].reset_index(drop=True), col_name, glb).iloc[glb-1:].reset_index(drop=True)
    Y = X[f"{frtype}{lf}"]
    X = X[X.columns[-1]]
    nmi = calc_NMI(X, Y)



0.1273916087404793
